<a href="https://colab.research.google.com/github/aliebi/Bert_tokenizer/blob/master/Bert_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT Dependencies

In [11]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [12]:
!pip install bert-for-tf2
!pip install sentencepiece


     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30539 sha256=01c4dc7626fc69025cd8c29f32f9ce32b88b35183ecc189abf2abc3617f2fa22
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=a52b89d2c29072e9c7752276c7599a41124a770d6f3c002af86d72a57bf2fb31
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=e10a9485fb6d2dd49dc541fc6e4ba837cd680bbdfa9b8b93252a2422a8d53dcb
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 5.8MB/s 


In [13]:
try : 
  %tensorflow_version 2.x
except Exception:

    pass

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert



Loding Data

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
cols = ["sentiment","id","date","query","user","text"]
data = pd.read_csv('/content/drive/MyDrive/Bert/train.csv',
                   header =None,
                   names=cols,
                   engine='python',
                   encoding='latin1')

In [16]:
data.head(5)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
data.drop(["id","date","query","user"],
          axis = 1,
          inplace = True)

In [18]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [19]:
data.text

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1600000, dtype: object

Preprocessing

In [20]:
#Cleaning
def clean_tweet(tweet):
    tweet =  BeautifulSoup(tweet,"lxml").get_text()
    tweet =  re.sub(r"@[A-Za-z0-9]+",' ',tweet)
    tweet =  re.sub(r"https?://[A-Za-z0-9./]+",' ',tweet)
    tweet =  re.sub(r"[^a-zA-Z.!?']",' ',tweet)
    tweet =  re.sub(r" +",' ',tweet)
    return tweet



In [21]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [22]:
data_clean

[" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!",
 ' I dived many times for the ball. Managed to save The rest go out of bounds',
 'my whole body feels itchy and like its on fire ',
 " no it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ",
 ' not the whole crew ',
 'Need a hug ',
 " hey long time no see! Yes.. Rains a bit only a bit LOL I'm fine thanks how's you ?",
 " K nope they didn't have it ",
 ' que me muera ? ',
 "spring break in plain city... it's snowing ",
 'I just re pierced my ears ',
 " I couldn't bear to watch it. And I thought the UA loss was embarrassing . . . . .",
 ' It it counts idk why I did either. you never talk to me anymore ',
 " i would've been the first but i didn't have a gun. not really though zac snyder's just a doucheclown.",
 ' I wish I got to watch it with you!! I miss you

In [23]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [24]:
data_labels

array([0, 0, 0, ..., 1, 1, 1])

Tokenization

In [25]:
FullTokenizer = bert.bert_tokenization.FullTokenizer 
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
 
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file,do_lower_case) 

In [26]:
tokenizer.tokenize("Hi dear , this is for testing applesss")

['hi', 'dear', ',', 'this', 'is', 'for', 'testing', 'apples', '##ss']

In [27]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Hi dear , this is for testing applesss"))

[7632, 6203, 1010, 2023, 2003, 2005, 5604, 18108, 4757]

In [28]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [29]:
 data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [30]:
data_inputs[0]

[22091,
 2860,
 2860,
 2008,
 1005,
 1055,
 1037,
 26352,
 5017,
 1012,
 2017,
 2323,
 2050,
 2288,
 2585,
 12385,
 1997,
 2353,
 2154,
 2000,
 2079,
 2009,
 1012,
 1040]

Dataset Creation

In [31]:
# for add padding we need length of sentences / we want add padding per bach
data_with_len = [[sent,data_labels[i],len(sent)]
                 for i,sent in enumerate(data_inputs)]

random.shuffle(data_with_len) 
data_with_len.sort(key=lambda x : x[2])

In [32]:
sorted_all = [(sent_lab[0],sent_lab[1])
                for sent_lab in data_with_len if sent_lab[2] > 2]

In [33]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32,tf.int32))

In [34]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([3666, 2508, 2829], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [35]:
Batch_size = 32
all_batch = all_dataset.padded_batch(Batch_size,padded_shapes=((None, ),()))


In [36]:
NB_batches = math.ceil(len(sorted_all) / Batch_size)
NB_batches_test = NB_batches // 10
all_batch.shuffle(NB_batches)
test_dataset = all_batch.take(NB_batches_test)
train_dataset = all_batch.skip(NB_batches_test)


Model Building

In [37]:
class DCNN(tf.keras.Model):

    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN,self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,emb_dim)
        
        self.bigram =  layers.Conv1D(filters=nb_filters,
                                     kernel_size=2,
                                     padding ='valid',
                                     activation='relu')
        self.trigram =  layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding ='valid',
                                     activation ='relu')
        self.fourgram =  layers.Conv1D(filters=nb_filters,
                                     kernel_size=4,
                                     padding='valid',
                                     activation='relu')
        
        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units=FFN_units,
                                    activation = "relu")
        self.dropout = layers.Dropout(rate=dropout_rate)

        if nb_classes ==2:
          self.last_dense = layers.Dense(units=1,
                                         activation="sigmoid")
          
        else :
          self.last_dense = layers.Dense(units=nb_classes,
                                         activation="softmax")
          
    def call(self,inputs,training):
        
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3) #(batch_size,nb_filters)

        merged = tf.concat([x_1,x_2,x_3], axis=-1) #(batch_size ,3*nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output





      

Training

In [38]:
vocab_size = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2
NB_EPOCHS = 1


In [39]:
Dcnn =   DCNN(vocab_size=vocab_size,
              emb_dim=EMB_DIM,
              nb_filters=NB_FILTERS,
              FFN_units=FFN_UNITS,
              nb_classes=NB_CLASSES,
              dropout_rate=DROPOUT_RATE)

In [40]:
 if NB_CLASSES == 2:
   Dcnn.compile(loss="binary_crossentropy",
                optimizer = "adam",
                metrics=["accuracy"])
   
else :
  Dcnn.compile(loss="sparse_categorical_crossentropy",
               optimizer="adam",
               metrics=["sparce_categorical_accuracy"])

In [41]:
checkpoint_path = "/content/drive/MyDrive/Bert"
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)
ckpt_manager = tf.train.CheckpointManager(ckpt,checkpoint_path,max_to_keep=1)

if ckpt_manager.latest_checkpoint :
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("latest checkpoint restored!")


latest checkpoint restored!


In [42]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self,epoch,logs =None):

      ckpt_manager.save()
      print("Checkpoint saved af {}.".format(checkpoint_path))

      

In [43]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks = [MyCustomCallback()])

  44240/Unknown - 2484s 56ms/step - loss: 0.2490 - accuracy: 0.8959Checkpoint saved af /content/drive/MyDrive/Bert.
44240/44240 [==============================] - 2484s 56ms/step - loss: 0.2490 - accuracy: 0.8959


Evaluation

In [44]:
Result = Dcnn.evaluate(test_dataset)
print(Result)

4915/4915 [==============================] - 26s 5ms/step - loss: 0.4663 - accuracy: 0.7621
[0.4662638306617737, 0.7620867490768433]


In [47]:
def get_prediction(sentence):
   token =  encode_sentence(sentence)
   inputs = tf.expand_dims(token,0)

   output = Dcnn(inputs,training = False)
   sentiment =  math.floor(output*2)

   if sentiment == 0:
     print("output of the model NEG: {} ".format(output))

   if sentiment == 1:
     print("output of the model POS: {} ".format(output))






In [56]:
get_prediction("this is too bad")

output of the model NEG: [[0.00290792]] 
